In [1]:
import sagemaker
import os

In [2]:
import pathlib as pl

In [3]:
import datetime

In [4]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import TensorBoardOutputConfig

In [5]:
MODEL = "MobileNetV2"
EPOCHS = 3
STEPS_PER_EPOCH = 1

In [6]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [8]:
?sess.upload_data

Signature: sess.upload_data(path, bucket=None, key_prefix='data', extra_args=None)
Docstring:
Upload local file or directory to S3.

If a single file is specified for upload, the resulting S3 object key is
``{key_prefix}/{filename}`` (filename does not include the local path, if any specified).

If a directory is specified for upload, the API uploads all content, recursively,
preserving relative structure of subdirectories. The resulting object key names are:
``{key_prefix}/{relative_subdirectory_path}/filename``.

Args:
    path (str): Path (absolute or relative) of local file or directory to upload.
    bucket (str): Name of the S3 Bucket to upload to (default: None). If not specified, the
        default bucket of the ``Session`` is used (if default bucket does not exist, the
        ``Session`` creates it).
    key_prefix (str): Optional S3 object key name prefix (default: 'data'). S3 uses the
        prefix to create a directory structure for the bucket content that it display in


In [9]:
upload_data = True
if upload_data:
    data_input_path = sess.upload_data(
        'obsoletely_data/split_images',
        key_prefix='obsoletely_data/split_images'
    )
else:
    data_input_path = "s3://sagemaker-us-east-2-475496805360/obsoletely_data"
print(data_input_path)

s3://sagemaker-us-east-2-475496805360/obsoletely_data/split_images


In [10]:
training_input_path = data_input_path + "/train"
validation_input_path = data_input_path + "/validate"

In [11]:
time_now = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%Mm%Ss")
tensorboard_logs_dir = f"s3://{sess.default_bucket()}/obsoletely_logs/{time_now}_{MODEL}"

In [12]:
tensorboard_output_config = TensorBoardOutputConfig(s3_output_path=tensorboard_logs_dir)

In [ ]:
tf_estimator = TensorFlow(
    entry_point='fine_tune_eff_net.py', 
    role=role,
    instance_count=1, 
    instance_type='ml.g4dn.xlarge',  # local 'ml.g4dn.xlarge'
    framework_version='2.3', 
    py_version='py37',
    script_mode=True,
    hyperparameters={
        'epochs': EPOCHS,
        "steps-per-epoch": STEPS_PER_EPOCH,
        "log-dir": tensorboard_logs_dir,
        "model": MODEL
    },
    tensorboard_output_config=tensorboard_output_config,
)

In [ ]:
tf_estimator.fit(
    {
        'training': training_input_path, 
        'validation': validation_input_path, 
    }
)